In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import time
import json
from RecurrentNeuralNetwork import RecurrentNeuralNetwork
from Torch_RNN import Torch_RNN
from torch.utils.data import DataLoader, TensorDataset

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.losses import MAE
from tensorflow.keras.layers import LSTM, GRU, Dropout, Dense

import matplotlib.pyplot as plt

In [2]:
class BaseRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, layers):
        super(BaseRNN, self).__init__()
        self.layers = layers

    def forward(self, x):
        pass

    def fit(self, X, y, batch_size=128, epochs=1, lr=0.01, val_split=0.2):
        X_train = X[: int((1-val_split) * len(X))]
        X_test  = X[: int(val_split * len(X))]
        y_train = y[: int((1-val_split) * len(y))]
        y_test  = y[: int(0.2 * len(y))]
        
        # Konvertieren der Daten in PyTorch-Tensoren
        X_train = torch.tensor(X_train, dtype=torch.float32)
        X_test = torch.tensor(X_test, dtype=torch.float32)
        y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
        y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

        # Erstellen von DataLoader-Objekten für das effiziente Laden von Daten während des Trainings
        batch_size = batch_size
        train_dataset = TensorDataset(X_train, y_train)
        test_dataset = TensorDataset(X_test, y_test)
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        # Definition von Verlust und Optimierer
        criterion = nn.L1Loss()  # Mean Absolute Error
        optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        
        train_loss_vals= []
        test_loss_vals = []
        
        
        for epoch in range(epochs):
            start_time = time.time()
            self.train(True)
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = self.forward(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

            # Ausgabe des Verlusts pro Epoche
            print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}, Duration: {round((time.time() - start_time)/60, 3)}')
            train_loss_vals.append(round(loss.item(),4))
            
            val_loss = 0.0
            self.eval()
            
            with torch.no_grad():
                start_time = time.time()
                for batch_X, batch_y in test_loader:
                    pred = self.forward(batch_X)
                    loss = criterion(pred, batch_y)
                    val_loss += loss
            
            val_loss = val_loss / batch_size
            #print(val_loss.shape)
            test_loss_vals.append(float(val_loss))
            print(f'Val Loss {val_loss}, Duration: {round((time.time() - start_time)/60, 3)}')
            
        return (train_loss_vals, test_loss_vals)

class SingleLayerRNN(BaseRNN):
    def __init__(self):
        super(SingleLayerRNN, self).__init__(input_size=30, hidden_size=30, output_size=1, layers=1)
        self.lstm1 = nn.RNN(30, 30, batch_first=True, bidirectional=False)
        self.linear1 = nn.Linear(30, 15)
        self.linear2 = nn.Linear(15, 1)

    def forward(self, x):
        out, _ = self.lstm1(x)
        out = self.linear1(out)
        out = self.linear2(out)
        return out

class DoubleLayerRNN(BaseRNN):
    def __init__(self):
        super(DoubleLayerRNN, self).__init__(input_size=30, hidden_size=30, output_size=1, layers=2)
        self.lstm1 = nn.RNN(30, 30, batch_first=True, bidirectional=False)
        self.dropout = nn.Dropout(0.2)
        self.lstm2 = nn.RNN(30, 30, batch_first=True, bidirectional=False)
        self.linear1 = nn.Linear(30, 15)
        self.linear2 = nn.Linear(15, 1)

    def forward(self, x):
        out, _ = self.lstm1(x)
        out = self.dropout(out)
        out, _ = self.lstm2(out)
        out = self.linear1(out)
        out = self.linear2(out)
        return out

class TripleLayerRNN(BaseRNN):
    def __init__(self):
        super(TripleLayerRNN, self).__init__(input_size=30, hidden_size=30, output_size=1, layers=3)
        self.lstm1 = nn.RNN(30, 30, batch_first=True, bidirectional=False)
        self.dropout1 = nn.Dropout(0.2)
        self.lstm2 = nn.RNN(30, 30, batch_first=True, bidirectional=False)
        self.dropout2 = nn.Dropout(0.2)
        self.lstm3 = nn.RNN(30, 30, batch_first=True, bidirectional=False)
        self.linear1 = nn.Linear(30, 15)
        self.linear2 = nn.Linear(15, 1)

    def forward(self, x):
        out, _ = self.lstm1(x)
        out = self.dropout1(out)
        out, _ = self.lstm2(out)
        out = self.dropout2(out)
        out, _ = self.lstm3(out)
        out = self.linear1(out)
        out = self.linear2(out)
        return out

def createRNN(layers=1):
    if layers == 1:
        model = SingleLayerRNN()
    elif layers == 2:
        model = DoubleLayerRNN()
    elif layers == 3:
        model = TripleLayerRNN()
    else:
        raise ValueError("Invalid number of layers. Supported values are 1, 2, or 3.")
    return model


In [3]:
#create functions that create the models

def createLSTM(layers=1, lr = 0.01):
    if layers == 1:
        model = Sequential()
        model.add(tf.keras.layers.LSTM(30, return_sequences=False, activation='tanh'))
        model.add(tf.keras.layers.Dense(15))
        model.add(tf.keras.layers.Dense(1))
        
    if layers == 2:
        model = Sequential()
        model.add(tf.keras.layers.LSTM(30, return_sequences=True, activation='tanh'))
        model.add(tf.keras.layers.Dropout(0.20))
        model.add(tf.keras.layers.LSTM(30, return_sequences=False, activation='tanh'))
        model.add(tf.keras.layers.Dense(15))
        model.add(tf.keras.layers.Dense(1))
        
    if layers == 3:
        model = Sequential()
        model.add(tf.keras.layers.LSTM(30, return_sequences=True, activation='tanh'))
        model.add(tf.keras.layers.Dropout(0.20))
        model.add(tf.keras.layers.LSTM(30, return_sequences=True, activation='tanh'))
        model.add(tf.keras.layers.Dropout(0.20))
        model.add(tf.keras.layers.LSTM(30, return_sequences=False, activation='tanh'))
        model.add(tf.keras.layers.Dense(15))
        model.add(tf.keras.layers.Dense(1))
        
    model.compile(loss = MAE, optimizer =Adam(learning_rate= lr),metrics=[MAE])
    return model


def createGRU(layers=1, lr = 0.01):
    if layers == 1:
        model = Sequential()
        model.add(tf.keras.layers.GRU(30, return_sequences=False, activation='tanh'))
        model.add(tf.keras.layers.Dense(15))
        model.add(tf.keras.layers.Dense(1))
        
    if layers == 2:
        model = Sequential()
        model.add(tf.keras.layers.GRU(30, return_sequences=True, activation='tanh'))
        model.add(tf.keras.layers.Dropout(0.20))
        model.add(tf.keras.layers.GRU(30, return_sequences=False, activation='tanh'))
        model.add(tf.keras.layers.Dense(15))
        model.add(tf.keras.layers.Dense(1))
        
    if layers == 3:
        model = Sequential()
        model.add(tf.keras.layers.GRU(30, return_sequences=True, activation='tanh'))
        model.add(tf.keras.layers.Dropout(0.20))
        model.add(tf.keras.layers.GRU(30, return_sequences=True, activation='tanh'))
        model.add(tf.keras.layers.Dropout(0.20))
        model.add(tf.keras.layers.GRU(30, return_sequences=False, activation='tanh'))
        model.add(tf.keras.layers.Dense(15))
        model.add(tf.keras.layers.Dense(1))
        
        
    model.compile(loss = MAE, optimizer =Adam(learning_rate= lr),metrics=[MAE])
    return model
        

In [10]:
#load data

data = pd.read_parquet('train_data_sensor_fault.parquet.gzip').to_numpy().astype(np.float32)

data = data[: int(0.2 * len(data))]

target_idx = 17

l = [i for i in  range(0,31)];l.remove(target_idx)

X, y = data[:,l], data[:,target_idx]

val_split = 0.2


X_train = X[: int((1-val_split) * len(X))]
X_test  = X[int((1-val_split) * len(X)): ]
y_train = y[: int((1-val_split) * len(y))]
y_test  = y[int((1-val_split) * len(X)):]
        

1182643

In [5]:
layers = [1,2,3]

alpha = 0.01
batch_size = 256
epochs = 20


results = {
    'RNN': {
        'Train_Loss(MAE)' : [],
        'Val_Loss(MAE)' : []
    },
    'GRU': {
        'Train_Loss(MAE)' : [],
        'Val_Loss(MAE)' : []
    },
    'LSTM': {
        'Train_Loss(MAE)' : [],
        'Val_Loss(MAE)' : []
    }
}

for i in [1,2,3]:
    LSTM = createLSTM(layers = i, lr=alpha)
    GRU = createGRU(layers = i, lr=alpha)
    RNN = createRNN(layers = i)
    
    
    tloss, vloss = RNN.fit(X, y, batch_size=batch_size, epochs=epochs, lr=alpha, val_split = 0.2)
    LSTM_hist = LSTM.fit(np.array(X).reshape(X.shape[0], X.shape[1], 1), np.array(y).reshape(y.shape[0],1), validation_split=0.2, epochs = epochs, batch_size = batch_size)
    GRU_hist = GRU.fit(np.array(X).reshape(X.shape[0], X.shape[1], 1), np.array(y).reshape(y.shape[0],1) , validation_split=0.2, epochs = epochs, batch_size = batch_size)
    
    results['RNN']['Train_Loss(MAE)'].append(tloss)
    results['RNN']['Val_Loss(MAE)'].append(vloss)
    
    results['LSTM']['Train_Loss(MAE)'].append(LSTM_hist.history['loss'])
    results['LSTM']['Val_Loss(MAE)'].append(LSTM_hist.history['val_mean_absolute_error'])
    
    results['GRU']['Train_Loss(MAE)'].append(GRU_hist.history['loss'])
    results['GRU']['Val_Loss(MAE)'].append(GRU_hist.history['val_mean_absolute_error'])

    
with open("Architecture_resuts.json", "w") as outfile: 
        json.dump(results, outfile)

2023-12-06 20:19:17.726311: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2023-12-06 20:19:17.726333: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2023-12-06 20:19:17.726338: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2023-12-06 20:19:17.726368: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-06 20:19:17.726382: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/20, Loss: 0.2641478478908539, Duration: 0.564
Val Loss 1.76601243019104, Duration: 0.035
Epoch 2/20, Loss: 0.2487592250108719, Duration: 0.567
Val Loss 3.181192398071289, Duration: 0.034
Epoch 3/20, Loss: 0.2350243777036667, Duration: 0.584
Val Loss 1.8374286890029907, Duration: 0.036
Epoch 4/20, Loss: 0.2414693385362625, Duration: 0.576
Val Loss 1.965866208076477, Duration: 0.034
Epoch 5/20, Loss: 0.2643311321735382, Duration: 0.576
Val Loss 3.7447350025177, Duration: 0.035
Epoch 6/20, Loss: 0.27503296732902527, Duration: 0.564
Val Loss 2.2373030185699463, Duration: 0.035
Epoch 7/20, Loss: 0.4348088204860687, Duration: 0.574
Val Loss 2.0735316276550293, Duration: 0.034
Epoch 8/20, Loss: 0.38661015033721924, Duration: 0.569
Val Loss 2.1987500190734863, Duration: 0.035
Epoch 9/20, Loss: 0.23754312098026276, Duration: 0.571
Val Loss 2.0571420192718506, Duration: 0.034
Epoch 10/20, Loss: 0.23915240168571472, Duration: 0.572
Val Loss 1.8463208675384521, Duration: 0.035
Epoch 11/20,

2023-12-06 20:31:25.078249: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-06 20:31:25.258805: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-06 20:31:26.594000: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4620/4620 [==============================] - ETA: 0s - loss: 0.5068 - mean_absolute_error: 0.5068

2023-12-06 20:32:23.472516: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-06 20:32:23.539723: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


...

4620/4620 [==============================] - 118s 24ms/step - loss: 0.4797 - mean_absolute_error: 0.4797 - val_loss: 0.2451 - val_mean_absolute_error: 0.2451
Epoch 2/20
4620/4620 [==============================] - 106s 23ms/step - loss: 0.2824 - mean_absolute_error: 0.2824 - val_loss: 0.1351 - val_mean_absolute_error: 0.1351
Epoch 3/20
4620/4620 [==============================] - 106s 23ms/step - loss: 0.2551 - mean_absolute_error: 0.2551 - val_loss: 0.2288 - val_mean_absolute_error: 0.2288
Epoch 4/20
4620/4620 [==============================] - 107s 23ms/step - loss: 0.2294 - mean_absolute_error: 0.2294 - val_loss: 0.0416 - val_mean_absolute_error: 0.0416
Epoch 5/20
4620/4620 [==============================] - 107s 23ms/step - loss: 0.2152 - mean_absolute_error: 0.2152 - val_loss: 0.2471 - val_mean_absolute_error: 0.2471
Epoch 6/20
4620/4620 [==============================] - 106s 23ms/step - loss: 0.2293 - mean_absolute_error: 0.2293 - val_loss: 0.4362 - val_mean_absolute_error: 0.43

2023-12-06 23:40:26.066545: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-06 23:40:26.303281: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-06 23:40:26.884734: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-06 23:40:27.485362: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-06 23:40:28.572773: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-06 23:40:29.593353: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-06 23:40:30.606892: I tensorflow/core/grappler/optimizers/cust

4620/4620 [==============================] - ETA: 0s - loss: 0.4434 - mean_absolute_error: 0.4434

2023-12-06 23:42:07.115273: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-06 23:42:07.214039: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-06 23:42:07.567432: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-06 23:42:08.029311: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4620/4620 [==============================] - 113s 23ms/step - loss: 0.4434 - mean_absolute_error: 0.4434 - val_loss: 0.1874 - val_mean_absolute_error: 0.1874
Epoch 2/20
4620/4620 [==============================] - 102s 22ms/step - loss: 0.2298 - mean_absolute_error: 0.2298 - val_loss: 0.0239 - val_mean_absolute_error: 0.0239
Epoch 3/20
4620/4620 [==============================] - 102s 22ms/step - loss: 0.2170 - mean_absolute_error: 0.2170 - val_loss: 0.0422 - val_mean_absolute_error: 0.0422
Epoch 4/20
4620/4620 [==============================] - 103s 22ms/step - loss: 0.2131 - mean_absolute_error: 0.2131 - val_loss: 0.4335 - val_mean_absolute_error: 0.4335
Epoch 5/20
4620/4620 [==============================] - 103s 22ms/step - loss: 0.2223 - mean_absolute_error: 0.2223 - val_loss: 0.1928 - val_mean_absolute_error: 0.1928
Epoch 6/20
4620/4620 [==============================] - 102s 22ms/step - loss: 0.2014 - mean_absolute_error: 0.2014 - val_loss: 0.4685 - val_mean_absolute_error: 0.46

In [6]:
#track scores for different lr

batch_size = 256
epochs = 20


results = {
    'RNN': {
        'Train_Loss(MAE)' : [],
        'Val_Loss(MAE)' : []
    },
    'GRU': {
        'Train_Loss(MAE)' : [],
        'Val_Loss(MAE)' : []
    },
    'LSTM': {
        'Train_Loss(MAE)' : [],
        'Val_Loss(MAE)' : []
    }
}

for lr in [0.05, 0.01, 0.005, 0.001]:
    LSTM = createLSTM(layers = 2, lr=lr)
    GRU = createGRU(layers = 2, lr=lr)
    RNN = createRNN(layers = 2)
    
    
    tloss, vloss = RNN.fit(X, y, batch_size=batch_size, epochs=epochs, lr=lr, val_split = 0.2)
    LSTM_hist = LSTM.fit(np.array(X).reshape(X.shape[0], X.shape[1], 1), np.array(y).reshape(y.shape[0],1), validation_split=0.2, epochs = epochs, batch_size = batch_size)
    GRU_hist = GRU.fit(np.array(X).reshape(X.shape[0], X.shape[1], 1), np.array(y).reshape(y.shape[0],1) , validation_split=0.2, epochs = epochs, batch_size = batch_size)
    
    results['RNN']['Train_Loss(MAE)'].append(tloss)
    results['RNN']['Val_Loss(MAE)'].append(vloss)
    
    results['LSTM']['Train_Loss(MAE)'].append(LSTM_hist.history['loss'])
    results['LSTM']['Val_Loss(MAE)'].append(LSTM_hist.history['val_mean_absolute_error'])
    
    results['GRU']['Train_Loss(MAE)'].append(GRU_hist.history['loss'])
    results['GRU']['Val_Loss(MAE)'].append(GRU_hist.history['val_mean_absolute_error'])

with open("LR_resuts.json", "w") as outfile: 
        json.dump(results, outfile)

Epoch 1/20, Loss: 0.25999051332473755, Duration: 1.073
Val Loss 2.0682857036590576, Duration: 0.057
Epoch 2/20, Loss: 0.23664772510528564, Duration: 1.071
Val Loss 2.7852604389190674, Duration: 0.056
Epoch 3/20, Loss: 0.3111225366592407, Duration: 1.072
Val Loss 2.1232876777648926, Duration: 0.056
Epoch 4/20, Loss: 0.2588888704776764, Duration: 1.071
Val Loss 2.3015921115875244, Duration: 0.057
Epoch 5/20, Loss: 0.25753045082092285, Duration: 1.071
Val Loss 1.7375409603118896, Duration: 0.056
Epoch 6/20, Loss: 0.2360737919807434, Duration: 1.071
Val Loss 1.8079088926315308, Duration: 0.056
Epoch 7/20, Loss: 0.24434508383274078, Duration: 1.072
Val Loss 1.923452377319336, Duration: 0.057
Epoch 8/20, Loss: 0.5135745406150818, Duration: 1.07
Val Loss 2.7574362754821777, Duration: 0.056
Epoch 9/20, Loss: 0.23351603746414185, Duration: 1.074
Val Loss 1.7568823099136353, Duration: 0.057
Epoch 10/20, Loss: 0.24329276382923126, Duration: 1.072
Val Loss 3.4636595249176025, Duration: 0.055
Epoch

2023-12-07 00:37:13.759329: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-07 00:37:13.938155: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-07 00:37:14.614736: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-07 00:37:15.743191: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-07 00:37:17.080899: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4620/4620 [==============================] - ETA: 0s - loss: 0.6252 - mean_absolute_error: 0.6252

2023-12-07 00:38:38.743407: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-07 00:38:38.796617: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-07 00:38:39.248104: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4620/4620 [==============================] - 95s 19ms/step - loss: 0.6252 - mean_absolute_error: 0.6252 - val_loss: 0.3617 - val_mean_absolute_error: 0.3617
Epoch 2/20
4620/4620 [==============================] - 86s 19ms/step - loss: 0.3698 - mean_absolute_error: 0.3698 - val_loss: 0.2425 - val_mean_absolute_error: 0.2425
Epoch 3/20
4620/4620 [==============================] - 86s 19ms/step - loss: 0.4108 - mean_absolute_error: 0.4108 - val_loss: 0.1932 - val_mean_absolute_error: 0.1932
Epoch 4/20
4620/4620 [==============================] - 86s 19ms/step - loss: 0.3524 - mean_absolute_error: 0.3524 - val_loss: 0.3031 - val_mean_absolute_error: 0.3031
Epoch 5/20
4620/4620 [==============================] - 86s 19ms/step - loss: 0.3355 - mean_absolute_error: 0.3355 - val_loss: 0.4542 - val_mean_absolute_error: 0.4542
Epoch 6/20
4620/4620 [==============================] - 86s 19ms/step - loss: 0.3209 - mean_absolute_error: 0.3209 - val_loss: 0.1989 - val_mean_absolute_error: 0.1989
Epo

...

4620/4620 [==============================] - 95s 19ms/step - loss: 1.3082 - mean_absolute_error: 1.3082 - val_loss: 0.0368 - val_mean_absolute_error: 0.0368
Epoch 2/20
4620/4620 [==============================] - 86s 19ms/step - loss: 0.0876 - mean_absolute_error: 0.0876 - val_loss: 0.0349 - val_mean_absolute_error: 0.0349
Epoch 3/20
4620/4620 [==============================] - 86s 19ms/step - loss: 0.0829 - mean_absolute_error: 0.0829 - val_loss: 0.0149 - val_mean_absolute_error: 0.0149
Epoch 4/20
4620/4620 [==============================] - 86s 19ms/step - loss: 0.0816 - mean_absolute_error: 0.0816 - val_loss: 0.0150 - val_mean_absolute_error: 0.0150
Epoch 5/20
4620/4620 [==============================] - 86s 19ms/step - loss: 0.0791 - mean_absolute_error: 0.0791 - val_loss: 0.0395 - val_mean_absolute_error: 0.0395
Epoch 6/20
4620/4620 [==============================] - 86s 19ms/step - loss: 0.0793 - mean_absolute_error: 0.0793 - val_loss: 0.0410 - val_mean_absolute_error: 0.0410
Epo

2023-12-07 05:01:07.926952: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-07 05:01:08.069530: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-07 05:01:08.790793: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-07 05:01:09.960308: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-07 05:01:11.191863: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4620/4620 [==============================] - ETA: 0s - loss: 1.1391 - mean_absolute_error: 1.1391

2023-12-07 05:02:27.299592: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-07 05:02:27.347892: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-07 05:02:27.779940: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4620/4620 [==============================] - 89s 18ms/step - loss: 1.1391 - mean_absolute_error: 1.1391 - val_loss: 0.0104 - val_mean_absolute_error: 0.0104
Epoch 2/20
4620/4620 [==============================] - 81s 18ms/step - loss: 0.0897 - mean_absolute_error: 0.0897 - val_loss: 0.0086 - val_mean_absolute_error: 0.0086
Epoch 3/20
4620/4620 [==============================] - 81s 18ms/step - loss: 0.0842 - mean_absolute_error: 0.0842 - val_loss: 0.0533 - val_mean_absolute_error: 0.0533
Epoch 4/20
4620/4620 [==============================] - 81s 18ms/step - loss: 0.0832 - mean_absolute_error: 0.0832 - val_loss: 0.0265 - val_mean_absolute_error: 0.0265
Epoch 5/20
4620/4620 [==============================] - 80s 17ms/step - loss: 0.0811 - mean_absolute_error: 0.0811 - val_loss: 0.0182 - val_mean_absolute_error: 0.0182
Epoch 6/20
4620/4620 [==============================] - 80s 17ms/step - loss: 0.0816 - mean_absolute_error: 0.0816 - val_loss: 0.0251 - val_mean_absolute_error: 0.0251
Epo

In [7]:
#track scores for different batch_size with fixed architecture and fixed lr

epochs = 20


results = {
    'RNN': {
        'Train_Loss(MAE)' : [],
        'Val_Loss(MAE)' : []
    },
    'GRU': {
        'Train_Loss(MAE)' : [],
        'Val_Loss(MAE)' : []
    },
    'LSTM': {
        'Train_Loss(MAE)' : [],
        'Val_Loss(MAE)' : []
    }
}

for batch_size in [32,64, 128, 256, 512]:
    LSTM = createLSTM(layers = 2, lr=0.01)
    GRU = createGRU(layers = 2, lr=0.01)
    RNN = createRNN(layers = 2)
    
    
    tloss, vloss = RNN.fit(X, y, batch_size=batch_size, epochs=epochs, lr=0.01, val_split = 0.2)
    LSTM_hist = LSTM.fit(np.array(X).reshape(X.shape[0], X.shape[1], 1), np.array(y).reshape(y.shape[0],1), validation_split=0.2, epochs = epochs, batch_size = batch_size)
    GRU_hist = GRU.fit(np.array(X).reshape(X.shape[0], X.shape[1], 1), np.array(y).reshape(y.shape[0],1) , validation_split=0.2, epochs = epochs, batch_size = batch_size)
    
    results['RNN']['Train_Loss(MAE)'].append(tloss)
    results['RNN']['Val_Loss(MAE)'].append(vloss)
    
    results['LSTM']['Train_Loss(MAE)'].append(LSTM_hist.history['loss'])
    results['LSTM']['Val_Loss(MAE)'].append(LSTM_hist.history['val_mean_absolute_error'])
    
    results['GRU']['Train_Loss(MAE)'].append(GRU_hist.history['loss'])
    results['GRU']['Val_Loss(MAE)'].append(GRU_hist.history['val_mean_absolute_error'])

with open("BatchSize_resuts.json", "w") as outfile: 
        json.dump(results, outfile)

Epoch 1/20, Loss: 0.42301639914512634, Duration: 1.352
Val Loss 145.02310180664062, Duration: 0.069
Epoch 2/20, Loss: 1.0389375686645508, Duration: 1.353
Val Loss 275.8783874511719, Duration: 0.07
Epoch 3/20, Loss: 0.761674165725708, Duration: 1.351
Val Loss 207.74595642089844, Duration: 0.069
Epoch 4/20, Loss: 0.4018412232398987, Duration: 1.352
Val Loss 137.68447875976562, Duration: 0.069
Epoch 5/20, Loss: 1.039750099182129, Duration: 1.347
Val Loss 282.1190490722656, Duration: 0.069
Epoch 6/20, Loss: 0.8196573853492737, Duration: 1.351
Val Loss 218.0937957763672, Duration: 0.07
Epoch 7/20, Loss: 0.8409359455108643, Duration: 1.356
Val Loss 221.93106079101562, Duration: 0.069
Epoch 8/20, Loss: 0.21105073392391205, Duration: 1.353
Val Loss 120.62797546386719, Duration: 0.069
Epoch 9/20, Loss: 0.6493955850601196, Duration: 1.352
Val Loss 172.7918243408203, Duration: 0.069
Epoch 10/20, Loss: 1.2848901748657227, Duration: 1.357
Val Loss 354.63299560546875, Duration: 0.069
Epoch 11/20, Lo

2023-12-07 05:56:33.197267: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-07 05:56:33.458422: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-07 05:56:34.192975: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-07 05:56:35.831138: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-07 05:56:37.347686: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


36958/36958 [==============================] - ETA: 0s - loss: 0.2580 - mean_absolute_error: 0.2580

2023-12-07 06:07:24.214022: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-07 06:07:24.268636: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-07 06:07:24.767069: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


36958/36958 [==============================] - 703s 19ms/step - loss: 0.2580 - mean_absolute_error: 0.2580 - val_loss: 0.0270 - val_mean_absolute_error: 0.0270
Epoch 2/20
36958/36958 [==============================] - 690s 19ms/step - loss: 0.1721 - mean_absolute_error: 0.1721 - val_loss: 0.0633 - val_mean_absolute_error: 0.0633
Epoch 3/20
36958/36958 [==============================] - 689s 19ms/step - loss: 0.1535 - mean_absolute_error: 0.1535 - val_loss: 0.1339 - val_mean_absolute_error: 0.1339
Epoch 4/20
36958/36958 [==============================] - 691s 19ms/step - loss: 0.1381 - mean_absolute_error: 0.1381 - val_loss: 0.0366 - val_mean_absolute_error: 0.0366
Epoch 5/20
36958/36958 [==============================] - 697s 19ms/step - loss: 0.1344 - mean_absolute_error: 0.1344 - val_loss: 0.1634 - val_mean_absolute_error: 0.1634
Epoch 6/20
36958/36958 [==============================] - 697s 19ms/step - loss: 0.1287 - mean_absolute_error: 0.1287 - val_loss: 0.0410 - val_mean_absolute

2023-12-07 09:47:00.525908: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-07 09:47:00.666577: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-07 09:47:01.399270: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-07 09:47:02.651453: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-07 09:47:03.918536: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


36958/36958 [==============================] - ETA: 0s - loss: 0.3368 - mean_absolute_error: 0.3368

2023-12-07 09:56:44.539759: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-07 09:56:44.590227: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-07 09:56:45.038793: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


36958/36958 [==============================] - 633s 17ms/step - loss: 0.3368 - mean_absolute_error: 0.3368 - val_loss: 0.2514 - val_mean_absolute_error: 0.2514
Epoch 2/20
36958/36958 [==============================] - 642s 17ms/step - loss: 0.2654 - mean_absolute_error: 0.2654 - val_loss: 0.1943 - val_mean_absolute_error: 0.1943
Epoch 3/20
36958/36958 [==============================] - 633s 17ms/step - loss: 0.2901 - mean_absolute_error: 0.2901 - val_loss: 0.2505 - val_mean_absolute_error: 0.2505
Epoch 4/20
36958/36958 [==============================] - 621s 17ms/step - loss: 0.2791 - mean_absolute_error: 0.2791 - val_loss: 0.2077 - val_mean_absolute_error: 0.2077
Epoch 5/20
36958/36958 [==============================] - 624s 17ms/step - loss: 0.2767 - mean_absolute_error: 0.2767 - val_loss: 0.2019 - val_mean_absolute_error: 0.2019
Epoch 6/20
36958/36958 [==============================] - 620s 17ms/step - loss: 0.2766 - mean_absolute_error: 0.2766 - val_loss: 0.4089 - val_mean_absolute

KeyboardInterrupt: 